### Login

In [ ]:
# Nacho la tiene hecha

### Pide stock

In [ ]:
@app.route('/Stock', methods=['GET'])
def Stock():
    #Obtenemos el ID del Get
    ID = request.args.get('ID')

    #Buscamos en la BBDD el Stock relacionado con el ID
    sql = "SELECT Stock FROM Productos WHERE ID = '%s'" % (ID)
    rc,cursor=ejecuta_query(sql)
    if rc != 0:
        return jsonify({"Error": "Error al buscar en la BBDD"}), 500
    else:
        #Creamos una lista para guardar los datos
        lista = []
        for row in cursor:
            lista.append(row)
        return jsonify(lista), 200

### Pide stock (ver si hay suficiente stock) + Envío stock

In [ ]:
## Lo tiene hecho Nacho

### Tramita pedido

In [ ]:
@app.route('/tramita_pedido', methods=['POST'])
def enviar_datos():

    # Obtener los datos del formulario
    
    user_id = request.form['user_id']
    product_id = request.form['product_id']
    quantity = request.form['quantity']
    unit_price = request.form['unit_price']
    creation_date = request.form['creation_date']
    status = request.form['status']


    sql = "INSERT INTO Pedidos (User_ID, ID, CreationDate, Status) VALUES (%s, %s, %s, %s)"

    # El valor del ID del pedido se genera automáticamente en la BBDD, cogiendo el último valor y sumándole 1 a este.

    ejecutar_query("SELECT MAX(ID) FROM Pedidos")

    val = (user_id, id_pedido, creation_date, status)

    ejecutar_query(sql, val)

    

    comprobador_Pedido = True

    # Comprobar si los productos están en la BBDD COMPROBAR STOCK TODOS LOS 6.X

    ejecutar_query("SELECT * FROM Productos WHERE ID = %s", (product_id,))

    myresult = mycursor.fetchall()

    if len(myresult) == 0:
        return 'El producto no existe'
    else:
        sql = "INSERT INTO Productos (ID, Quantity, UnitPrice) VALUES (%s, %s, %s)"
        val = (product_id, quantity, unit_price)

        ejecutar_query(sql, val)

        

        comprobador_Producto = True

    # Crea un nuevo OrderDetail en la tabla OrderDetails con los datos correspondientes.

    sql = "INSERT INTO OrderDetails (Order_ID, Product_ID, Quantity, UnitPrice) VALUES (%s, %s, %s, %s)"

    # El valor del ID del pedido es el mismo que el ID del pedido que se ha creado en la tabla Pedidos.
    ejecutar_query("SELECT MAX(ID) FROM Pedidos")

    myresult = mycursor.fetchall()

    for x in myresult:
        id_pedido = x[0]

    val = (id_pedido, product_id, quantity, unit_price)

    ejecutar_query(sql, val)

    

### Añade pedido al almacén

In [ ]:
@app.route('/nuevo_pedido', methods=['POST'])
def nuevo_pedido():

    # El order_id se coge el último que exista en la BBDD de Pedidos

    ejecutar_query("SELECT MAX(Order_ID) FROM Pedidos")

    myresult = mycursor.fetchall()

    for x in myresult:
        order_id = x[0]
    

    # El estatus se actualiza en la BBDD del Pedido a "En almacén"

    ejecutar_query("UPDATE Pedidos SET Status = 'En almacen' WHERE ID = %s", (order_id,))

    

    sql = "INSERT INTO cola_almacen (Order_ID, Status) VALUES (%s, %s)"
    val = (order_id, status)

    ejecutar_query(sql, val)

    

    return 'Nuevo pedido confirmado'

## Enviar pedido a Picking y escanear productos

In [ ]:
@app.route('/pedir_pedido', methods=['POST'])
def pedir_pedido():
    # Recibe los datos Order_ID y Status de cola_almacen, y el PickingStation_ID, date de picking_records
    picking_station_id = request.json['PickingStation_ID']
    
    # Utilizando el picking_station_id, obtengo order_id, status y date de picking_records, sabiendo que esa tabla tiene como atributos PickingRecord_ID, Picking_Statin_ID, Order_ID, Date y Status

    

    ejecutar_query("SELECT * FROM picking_records WHERE PickingStation_ID = %s", (picking_station_id,))


    
    # Actualizo la tabla cola_almacen con los datos obtenidos de picking_records, sabiendo que la tabla cola_almacen tiene como atributos Queue_ID, Order_ID y Status

    ejecutar_query("SELECT * FROM cola_almacen WHERE Order_ID = %s", (order_id,))


    sql = "UPDATE cola_almacen SET Status = %s WHERE Queue_ID = %s"
    val = (status, queue_id)

    ejecutar_query(sql, val)

    

    return 'Datos enviados a la cola del almacen'

@app.route('/envia_pedido', methods=['POST'])
def envia_pedido():

    # Coge el último pedido de la cola_almacen y lo envías a puesto_picking

    ejecutar_query("SELECT MAX(Order_ID) FROM cola_almacen")

    myresult = mycursor.fetchall()



    # Obtenemos el PickingStation_ID

    ejecutar_query("SELECT MAX(PickingStation_ID) FROM Puesto_picking")

    myresult = mycursor.fetchall()

    for x in myresult:
        id_puestoPicking = x[0]

    # Asignamos ese pedido a un PickingRecord con el OrderID 

    ejecutar_query("SELECT * FROM pickingRecord WHERE Order_ID = %s", (order_id,))
    myresult = mycursor.fetchall()

    if len(myresult) != 0:
        return 'El pedido ya existe en la cola del puesto de picking'
    else:
        ejecutar_query("INSERT into PickingRecord VALUES (%s, %s)")
        val = (order_id, id_puestoPicking)

        ejecutar_query(sql, val)

        

        # Actualizamos el status del pedido

        ejecutar_query("UPDATE Pedidos SET Status = 'En almacen' WHERE ID = %s", (order_id,))

        



        return 'Pedido enviado al puesto de picking'
    
@app.route('/escanea_producto', methods=['POST'])
def escanea_producto():

    # Función que simule el que se estén escanando los pedidos en el puesto picking
    # correspondiente al último order que exista

    # Una vez termine, indicar que el pedido se ha terminado de preparar

## Contactar con transportes

In [ ]:
# Modificar para que sea la que indique el parámetro "Transport" de la tabla pedidos

def obtener_informacion_envio(order_id):
    url_pedidos = "http://localhost:5000/pedidos"
    url_etiquetas = "http://localhost:5000/etiquetas_envio"
    headers = {'Content-Type': 'application/json'}

    # Una vez el pedido está terminado, esta función se llama

    # Obtenemos el ID del pedido, y dependiendo de las condiciones, se le asigna un tipo de envío u otro

    '''
    
    En el alcance incial se establecía como una de las salidas:
 La aplicación generará una etiqueta de envío con ID PEDIDO, nombre y dirección.
 La aplicación generará un albarán con referencias y cantidades reales enviadas.
La necesidad de integrarse con empresas de transporte requiere una pequeña modificación en esta salida,
tendrá que poder seleccionarse:
 Tipo de envío: estándar o urgente.
 Agencia de transporte: Correos, Seur o DHL.
Esta información deberá reflejarse en la etiqueta de envío de la siguiente manera:
 DHL
o Día y hora de pedido.
o Tipo de envío.
o ID de pedido, nombre y dirección.
o Listado de artículos reales a enviar.
 SEUR
o Día de entrega (estándar día de recogida +3, urgente día de recogida +1).
o Tipo de envío
o ID de pedido, nombre y dirección.
o Peso total del pedido (se estima 200gr por cada unidad).
 CORREOS
o Día de pedido.
o Tipo de envío.
o ID de pedido, nombre y dirección.
o Código postal de la entrega.
o Peso total del pedido (se estima 200gr por cada unidad).
o Cantidad en unidades total en la entrega.

    
    '''
    # Comprobar si la agencia es seur, dhl o correos

    

    informacion_envio = {
        'Order_ID': pedido['Order_ID'],
        'User_ID': pedido['User_ID'],
        'CreationDate': pedido['CreationDate'],
        'Status': pedido['Status'],
        'RecipientName': etiquetas['RecipientName'],
        'RecipientPhone': etiquetas['RecipientPhone'],
        'AddressLine1': etiquetas['AddressLine1'],
        'AddressLine2': etiquetas['AddressLine2'],
        'City': etiquetas['City'],
        'State_Province': etiquetas['State_Province'],
        'PostalCode': etiquetas['PostalCode'],
        'Country': etiquetas['Country'],
        'Transport': etiquetas['Transport']
    }

    

    # Imprimir información de envío QR
    print(informacion_envio)

    # Notificar usuario y cambiar STATUS